In [2]:
import polars as pl
import os
import duckdb as db

# Chemin data

In [3]:
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
data = os.path.join(parent_dir, "data")
data_path = os.path.join(data, "BPE23.parquet")

# BDD

In [4]:
df = pl.scan_parquet(data_path)

In [45]:
df_filtered = df.filter(
    (pl.col("DOM") == "D"), 
    (pl.col("TYPEQU").str.contains("D")),
    (pl.col("SDOM").is_in(["D1", "D2", "D3"])) |
    (pl.col("ACCES_SANITAIRE") == "1") |
    (pl.col("PRES_SANITAIRE") == "1")
).select([
    "DOM", "SDOM", "TYPEQU", "CAPACITE", "ACCES_SANITAIRE", "PRES_SANITAIRE",
    "DENS3", "DENS7", "AAV2020", "QP", "QP2015", "LATITUDE", "LONGITUDE",
    "CODPOS", "LIBCOM"
]).collect()


In [48]:
df_filtered["TYPEQU"].unique().sort().to_list()

['D101',
 'D102',
 'D103',
 'D104',
 'D105',
 'D106',
 'D107',
 'D108',
 'D109',
 'D110',
 'D111',
 'D112',
 'D113',
 'D201',
 'D202',
 'D203',
 'D206',
 'D207',
 'D208',
 'D209',
 'D210',
 'D211',
 'D212',
 'D213',
 'D214',
 'D221',
 'D231',
 'D233',
 'D235',
 'D236',
 'D237',
 'D238',
 'D239',
 'D240',
 'D242',
 'D243',
 'D244',
 'D302',
 'D303',
 'D304',
 'D305',
 'D307']

Les équipements de santé sont dans les variables :
    - DOM : D
    - SDOM : D1, D2, D3
    - TYPEQU : D101, D102, D103, D108, D113

variables importantes :
    - DOM  
    - SDOM 
    - TYPEQU 
    - CAPACITE 
    - ACCES_SANITAIRE 
    - PRES_SANITAIRE 
    - DENS3 
    - DENS7
    - AAV2020 
    - QP & QP2015
    - LATITUDE & LONGITUDE 
    - CODPOS 
    - LIBCOM 

# DuckDB

In [4]:
df = pl.scan_parquet(data_path)

In [5]:
df_filtered = df.select([
    "DOM", "SDOM", "TYPEQU", "CAPACITE", "ACCES_SANITAIRE", "PRES_SANITAIRE",
    "DENS3", "DENS7", "AAV2020", "QP", "QP2015", "LATITUDE", "LONGITUDE",
    "CODPOS", "LIBCOM"
])


In [6]:
df_filtered = df_filtered.collect()

In [7]:
con = db.connect(database=':memory:')
con.register('df', df_filtered)

In [8]:
query = """
SELECT *
FROM df
WHERE
    DOM = 'D'
    OR SDOM IN ('D1', 'D2', 'D3')
    OR TYPEQU IN ('D101', 'D102', 'D103', 'D108', 'D113')
    OR ACCES_SANITAIRE = '1'
    OR PRES_SANITAIRE = '1'
"""

In [9]:
df_sante = con.execute(query).fetchdf()

: 